In [1]:
import pathlib
import re


def rst2md(path_rst, path_md):
    title_marks = ['=', '-', '`', ':', "'", '"', '~', '^', '_', '*', '+', '#', '<', '>']
    appeared_title_marks = {}
    content = ''
    last_line = ''
    count = 0
    code_flag = image_flag = False
    with open(path_rst, 'r', encoding='utf-8') as f:
        for line in f:
            # 匹配标题
            if re.match(r"^[=\-`:'\"~^_*+#<>]{3,}", line) and not code_flag:
                if line[0] not in appeared_title_marks:
                    count += 1
                    appeared_title_marks[line[0]] = count
                if last_line:
                    last_line = '#' * appeared_title_marks[line[0]] + ' ' + last_line
                line = ''
                
            # 匹配代码块
            if line.startswith('.. code:: '):
                code_flag = True
                code_block = line.replace('.. code:: ', '```')
                line = ''
                if image_flag:
                    image_flag = False
                    last_line = '![](' + image_url + ')\n'                    
            # 匹配图片链接
            elif line.startswith('.. image:: ') or line.startswith('.. figure:: '):
                image_flag = True
                image_url = line.replace('.. image:: ', '').replace('.. figure:: ', '').strip()
                line = ''
                if code_flag:
                    code_flag = False
                    last_line = code_block + '```\n'
                
            if code_flag:
                if line.startswith('  ')  or line.startswith('\t') or not line.strip():
                    code_block += line
                    line = ''
                else:
                    code_flag = False
                    last_line = code_block + '```\n'
                
            if image_flag:
                if (line.startswith('  ') or line.startswith('\t') or not line.strip()) and not code_flag:
                    line = ''
                else:
                    image_flag = False
                    last_line = '![](' + image_url + ')\n'                    
                    
            content += last_line
            last_line = line
            
        content += line
        
        if code_flag:
            content += code_block + '```\n'
        elif image_flag:
            content += '![](' + image_url + ')'
            
    with open(path_md, 'w', encoding='utf-8') as f:
        f.write(content)


In [2]:
path_work = pathlib.Path(r'D:\Python_Work\reading_notes_cleaner\unrelease\docs')

rst_files = path_work.glob('**/*.rst')

count = 0
for rst_file in rst_files:
    path_md = rst_file.with_suffix('.md')
    rst2md(rst_file, path_md)
    count +=1

print(f"成功转换{count}个.rst文件为.md文件。")


成功转换104个.rst文件为.md文件。
